In [115]:
import os
from threading import Lock
import json

class JSONDatabase:
    
    def __init__(self, filename):
        self._filelock = Lock()
        self._cache_storage = self._create_empty_storage()
        
        # input parameters
        self._filename = filename
        
        if not os.path.exists(self._filename):
            with open(self._filename, 'w') as f:
                f.write(json.dumps(self._cache_storage))
    
    def _create_empty_storage(self):
        return {}
    
    def load_storage(self):
        try:
            self._filelock.acquire() #
            
            with open(self._filename, 'r') as f:
                txt = f.read()
                if not txt:
                    self._cache_storage = self._create_empty_storage()
                else:
                    self._cache_storage = json.loads(txt)
            return True
            
        finally:
            self._filelock.release()
    
    def flush_storage(self):
        try:
            self._filelock.acquire() #
            
            with open(self._filename, 'w') as f:
                f.write(json.dumps(self._cache_storage))
            return True
            
        finally:
            self._filelock.release()
    
    def get_key(self, key):
        try:
            self._filelock.acquire() #
            
            return self._cache_storage[key]
        
        finally:
            self._filelock.release()
            
    def set_key(self, key, value):
        try:
            self._filelock.acquire() #
            
            self._cache_storage[key] = value
            return True
            
        finally:
            self._filelock.release()
    
    def delete_key(self, key):
        try:
            self._filelock.acquire() #
            
            self._cache_storage.pop(key)
            return True
            
        finally:
            self._filelock.release()
    
    def get_cache_storage(self):
        try:
            self._filelock.acquire() #
            
            return self._cache_storage
            
        finally:
            self._filelock.release()

In [123]:
import pandas as pd
import time

class Position:
    
    def __init__(self, ticker, items_amount, pos_type, enter_level, stop_loss_level, reenter_level, exit_level):
        self.params = {}
        
        self.params['ticker'] = ticker
        self.params['items_amount'] = items_amount
        self.params['pos_type'] = pos_type
        self.params['enter_level'] = enter_level
        self.params['stop_loss_level'] = stop_loss_level
        self.params['reenter_level'] = reenter_level
        self.params['exit_level'] = exit_level
        
        #new or exit
        self.params['status'] = 'new'

class CommandPanel:
    
    def __init__(self, cmd_db):
        self._db = cmd_db
    
    def gen_new_pos_key_prefix(self):
        return f'{t:22.12f}'.replace('.', '').replace(',', '')
    
    def add_new_position(self, ticker, items_amount, pos_type, enter_level=1, stop_loss_level=2.3, reenter_level=2, exit_level=1):
        self._db.load_storage()
        new_key = f'{ticker}_{self.gen_new_pos_key_prefix()}'
        self._db.set_key(new_key, Position(ticker, items_amount, pos_type, enter_level, stop_loss_level, reenter_level, exit_level).params)
        self._db.flush_storage()
        
    def delete_position(self, key):
        self._db.load_storage()
        self._db.delete_key(key)
        self._db.flush_storage()
    
    def get_all_positions_pd(self):
        self._db.load_storage()
        pos = self._db.get_cache_storage()
        res = pd.DataFrame()
        
        for p in pos:
            item = pos[p]
            item['key'] = p
            res = res.append(item, ignore_index=True)
        return res

In [124]:
cmd_db = JSONDatabase('cmd_db.json')
cmd = CommandPanel(cmd_db)

In [125]:
cmd.get_all_positions_pd()

""


In [126]:
cmd.add_new_position(ticker='ROSN', items_amount=2, pos_type='long')

In [127]:
cmd.get_all_positions_pd()

,enter_level,exit_level,items_amount,key,pos_type,reenter_level,status,stop_loss_level,ticker
0,1.0,1.0,2.0,ROSN_1607541199865460634232,long,2.0,new,2.3,ROSN


In [37]:
db = JSONDatabase('test.json')

In [38]:
db.set_key('test', {'a': 1, 'b': 2})

True

In [39]:
db._cache_storage

{'test': {'a': 1, 'b': 2}}

In [40]:
db.flush_storage()

True

In [41]:
db.load_storage()

True

In [42]:
db.get_cache_storage()

{'test': {'a': 1, 'b': 2}}

In [84]:
import time
t = time.time()
t, f'{t:22.12f}'.replace('.', '').replace(',', '')

(1607541199.8654606, '1607541199865460634232')

In [56]:
type(time.time())

float